# ⭐ Entrenador de Lora de Hollowstrawberry

Este colab viene de [esta guía](https://huggingface.co/hollowstrawberry/stable-diffusion-guide/blob/main/spanish.md#index). Te guiará para obtener tus imágenes y descripciones rápidamente, para luego usarlas para entrenar un Lora.

Basado en el trabajo de [Kohya_ss y Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb#scrollTo=-Z4w3lfFKLjr). ¡Gracias!

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |

In [ ]:
#@title ## 🚩 Empieza Aquí
import os
import re
import shutil
import zipfile
from google.colab import drive
from google.colab import output as console
from IPython.display import Markdown, display

if "model_url" in globals():
  old_model_url = model_url
else:
  old_model_url = None
if "dependencies_installed" not in globals():
  dependencies_installed = False
if "model_file" not in globals():
  model_file = None

#@markdown ### ▶️ Inicio
#@markdown El nombre de tu proyecto también es el nombre de la carpeta donde irán tus imágenes. No se permiten espacios.
nombre_proyecto = "" #@param {type:"string"}
project_name = nombre_proyecto
#@markdown Decidir el modelo base de entrenamiento. Los modelos por defecto producen los resultados más limpios y consistentes. Puedes cambiarlo por un modelo propio si lo deseas.
modelo_de_entrenamiento = "Anime (animefull-final-pruned-fp16.safetensors)" #@param ["Anime (animefull-final-pruned-fp16.safetensors)", "Fotorealismo (sd-v1-5-pruned-noema-fp16.safetensors)"]
opcional_modelo_propio = "" #@param {type:"string"}

if opcional_modelo_propio:
  model_url = opcional_modelo_propio
elif "Anime" in modelo_de_entrenamiento:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/animefull-final-pruned-fp16.safetensors"
else:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/sd-v1-5-pruned-noema-fp16.safetensors"

#@markdown ### ▶️ Archivos <p>
#@markdown Si usaste mi [preparador de Lora](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb#scrollTo=0BJTp5PVN_q-), estás listo. Sino, crea una carpeta en tu Google Drive así: `lora_training/datasets/nombre_proyecto` y llénala con tus imágenes y textos. Puedes descomprimir un archivo zip en los Extras más abajo. <p>
#@markdown La resolución de 512 es estándar en Stable Diffusion 1.5. No es necesario recortar o achicar, el proceso es automático.
resolucion = 512 #@param {type:"number"}
resolution = resolucion
#@markdown Esta opción va a voltear tus imágenes para así tener el doble y aprender mejor. <p>
#@markdown **Desactiva esto si te importan los elementos asimétricos en tu Lora.**
flip_aug = True #@param {type:"boolean"}
#@markdown Si tienes una palabra de activación al inicio de tus archivos de texto, aumenta este valor a 1.
keep_tokens = 1 #@param {type:"slider", min:0, max:5, step:1}
#@markdown Si usas tags de anime mantén esto activo.
shuffle_caption = True #@param {type:"boolean"}
caption_extension = ".txt"

#@markdown ### ▶️ Pasos <p>
#@markdown Tus imágenes se repetirán durante el entrenamiento. Recomiendo que tu cantidad de imágenes multiplicada por las repeticiones esté entre 200 y 400.
repeticiones = 10 #@param {type:"number"}
num_repeats = repeticiones
#@markdown Un epoch es una cantidad de pasos equivalente a tu cantidad de imágenes multiplicada por las repeticiones y dividido por el batch size. <p>
#@markdown Más epochs le dará mas tiempo a tu Lora para aprender y más opciones para que puedas comparar. Recomiendo entre 10 y 30 epochs, lo que debería resultar en 1000 a 6000 pasos totales. Verás tus pasos al empezar el entrenamiento.
epochs = 15 #@param {type:"number"}
max_train_epochs = epochs
guardar_cada_cuantos_epochs = 1 #@param {type:"number"}
save_every_n_epochs = guardar_cada_cuantos_epochs
if save_every_n_epochs < 1:
  save_every_n_epochs = max_train_epochs
#@markdown Puedes dejar el batch size así o si tienes muchas imágenes puedes aumentarlo.
batch_size = 2 #@param {type:"slider", min:1, max:8, step:1}
train_batch_size = batch_size
#@markdown Pasos de "calentamiento" durante el entrenamiento. Recomiendo dejarlo en 5%.
calentamiento = 0.05 #@param {type:"slider", min:0.0, max:0.5, step:0.01}
lr_warmup_ratio = calentamiento
lr_warmup_steps = 0

#@markdown ### ▶️ Entrenamiento
#@markdown La tasa de aprendizaje (unet) es lo más importante. Puedes dejarlo como está y probar. Si tienes muy pocas imágenes prueba 1e-3. Si tienes muchas imágenes y quieres entrenar más despacio prueba 1e-4.<p>
#@markdown El text encoder ayuda a tu Lora a aprender conceptos un poco mejor. Se recomienda la mitad o un quinto del unet.
aprendizaje_unet = 5e-4 #@param {type:"number"}
unet_lr = aprendizaje_unet
aprendizaje_text_encoder = 1e-4 #@param {type:"number"}
text_encoder_lr = aprendizaje_text_encoder
#@markdown El scheduler es el algoritmo matemático que guiará el entrenamiento. Para personajes recomiendo `cosine_with_restarts` con un valor de 3. Si no estás seguro ponlo en `constant` e ignora el valor.
scheduler = "cosine_with_restarts" #@param ["constant", "cosine", "cosine_with_restarts", "constant_with_warmup", "linear", "polynomial"]
lr_scheduler = scheduler
valor_de_scheduler = 3 #@param {type:"number"}
lr_scheduler_number = valor_de_scheduler
lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0
#@markdown Más dim equivale a un Lora más grande. Podrá aprender más como también aprender más basura. Antes se usaba el dim 128, pero ahora se recomienda 16 o 32 con una tasa de aprendizaje mayor. <p>
#@markdown El alpha se recomienda que sea igual o la mitad del dim, o 1. <p>
#@markdown Puedes dejar ambos como están.
network_dim = 16 #@param {type:"slider", min:1, max:128, step:1}
network_alpha = 8 #@param {type:"slider", min:1, max:128, step:1}

#@markdown ### ▶️ Tipo de Lora
#@markdown LoCon y LoHa son nuevos tipos de LoRA que ofrecen mayor aprendizaje. Si deseas experimentar con ellos hazlo aquí, de otra forma no te preocupes.
#@markdown Se dice que los LoCon son buenos para los estilos. Más información [aquí](https://github.com/KohakuBlueleaf/Lycoris). Necesitarás [esta extensión](https://github.com/KohakuBlueleaf/a1111-sd-webui-locon) para usarlos en webui.
#@markdown Nota: La `conv_compression` hace el archivo más pequeño pero puede tener resultados inesperados. <p>
#@markdown Valores recomendados:

#@markdown | type | network_dim | network_alpha | conv_dim | conv_alpha |
#@markdown | :---: | :---: | :---: | :---: | :---: |
#@markdown | LoRA | 32 | 16 | - | - |
#@markdown | LoCon | 16 | 8 | 8 | 1 |
#@markdown | LoHa | 8 | 4 | 4 | 1 |

lora_type = "LoRA" #@param ["LoRA", "LoCon Kohya", "LoCon Lycoris", "LoHa Lycoris"]
conv_dim = 8 #@param {type:"slider", min:1, max:64, step:1}
conv_alpha = 1 #@param {type:"slider", min:1, max:64, step:1}
conv_compression = False #@param {type:"boolean"}

network_module = "lycoris.kohya" if "Lycoris" in lora_type else "networks.lora"
network_args = None if lora_type == "LoRA" else [
  f"conv_dim={conv_dim}",
  f"conv_alpha={conv_alpha}",
]
if "Lycoris" in lora_type:
  network_args.append(f"algo={'loha' if 'LoHa' in network_args else 'lora'}")
  network_args.append(f"disable_conv_cp={str(not conv_compression)}")

#@markdown ### ▶️ Listo
#@markdown Ahora puedes correr esta celda apretando el botón circular a la izquierda. ¡Buena suerte!

root_dir = "/content"
deps_dir = os.path.join(root_dir,"deps")
repo_dir = os.path.join(root_dir,"kohya-trainer")

main_dir = os.path.join(root_dir,"drive/MyDrive/lora_training")
config_dir = os.path.join(main_dir,"config")
datasets_dir = os.path.join(main_dir,"datasets")
output_dir = os.path.join(main_dir,"output")
logging_dir = os.path.join(main_dir,"log")

images_folder = os.path.join(datasets_dir, project_name)
output_folder = os.path.join(output_dir, project_name)
config_folder = os.path.join(config_dir, project_name)

accelerate_config = os.path.join(repo_dir, "accelerate_config/config.yaml")
tools_dir = os.path.join(repo_dir,"tools")
finetune_dir = os.path.join(repo_dir,"finetune")

images = None
dataset_config_file = None
config_file = None
  
def clone_repo():
  os.chdir(root_dir)
  !git clone https://github.com/Linaqruf/kohya-trainer {repo_dir}
  os.chdir(repo_dir)
  !git reset --hard 78624a16498ebcb323963e49fbf470b997c848df

def install_ubuntu_deps(url, name, dst):
    os.chdir(repo_dir)
    !wget -q --show-progress {url}
    with zipfile.ZipFile(name, "r") as deps:
        deps.extractall(dst)
    !dpkg -i {dst}/*
    os.remove(name)
    shutil.rmtree(dst)

def install_dependencies():
  os.chdir(repo_dir)

  !pip -q install --upgrade -r requirements.txt
  !pip install xformers triton

  # patch kohya for minor stuff
  !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
  !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
  !sed -i 's/model_name + "."/model_name + "-{:02d}.".format(num_train_epochs)/g' train_network.py # name of the last epoch will match the rest

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config):
    write_basic_config(save_location=accelerate_config)

def validate_dataset():
  global images, lr_warmup_steps, lr_warmup_ratio
  supported_types = (".png", ".jpg", ".jpeg")

  print("\n💿 Revisando archivos...")
  if not project_name.strip():
    print("💥 Error: Por favor elige un nombre de proyecto.")
    return

  # this is starting to spaghetti
  if "override_dataset_config_file" in globals() and override_dataset_config_file:
    try:
      datconf = toml.load(override_dataset_config_file)
      datasets = {d["image_dir"]: d["num_repeats"] for d in datconf["datasets"][0]["subsets"]}
    except:
      print(f"💥 Error: Tu configuración de dataset propia es inválida!")
      return
    folders = datasets.keys()
    files = [f for folder in folders for f in os.listdir(folder)]
    images_repeats = {folder: (len([f for f in os.listdir(folder) if f.lower().endswith(supported_types)]), datasets[folder]) for folder in folders}
  else:
    folders = [images_folder]
    files = os.listdir(images_folder)
    images_repeats = {images_folder: (len([f for f in files if f.lower().endswith(supported_types)]), num_repeats)}

  for folder in folders:
    if not os.path.exists(folder):
      print(f"💥 Error: La carpeta {folder.replace('/content/drive/', '')} no existe.")
      return
  for folder, (img, rep) in images_repeats.items():
    if not img:
      print(f"💥 Error: La carpeta {folder.replace('/content/drive/', '')} está vacía.")
      return
  if not [txt for txt in files if txt.lower().endswith(".txt")]:
    print("💥 Error: No tienes archivos de texto junto a tus imágenes. Por favor describe al menos 1 de tus imágenes para continuar.")
    return
  for f in files:
    if not f.lower().endswith(".txt") and not f.lower().endswith(supported_types):
      print(f"💥 Error: Archivo inválido encontrado: \"{f}\". Abortando.")
      return

  pre_steps_per_epoch = sum(img*rep for (img, rep) in images_repeats.values())
  steps_per_epoch = pre_steps_per_epoch/train_batch_size
  total_steps = int(max_train_epochs*steps_per_epoch)
  lr_warmup_steps = int(total_steps * lr_warmup_ratio)

  for folder, (img, rep) in images_repeats.items():
    print("📁"+folder.replace("/content/drive/", ""))
    print(f"📈 Se encontraron {img} imágenes con {rep} repeticiones, resultando en {img*rep} pasos.")
  print(f"📉 Divide {pre_steps_per_epoch} pasos en {train_batch_size} batch size para resultar en {steps_per_epoch} pasos por epoch.")
  print(f"🔮 Habrá {max_train_epochs} epochs, para un total de {total_steps} pasos de entrenamiento.")

  if total_steps > 10000:
    print("💥 Error: Tienes muchos pasos. Probablemente te equivocaste. Abortando...") 
    return
  return True

def create_config():
  global dataset_config_file, config_file, model_file
  import toml

  os.makedirs(os.path.join(config_dir, project_name), exist_ok=True)
  dataset_config_file = os.path.join(config_dir, project_name, "dataset_config.toml")
  config_file = os.path.join(config_dir, project_name, "training_config.toml")

  if "override_config_file" in globals() and override_config_file:
    config_file = override_config_file
    print(f"⭕ Usando configuración propia {config_file}")
  else:
    config_dict = {
      "additional_network_arguments": {
        "unet_lr": unet_lr,
        "text_encoder_lr": text_encoder_lr,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_module": network_module,
        "network_args": network_args,
        "network_train_unet_only": True if text_encoder_lr == 0 else None
      },
      "optimizer_arguments": {
        "learning_rate": unet_lr,
        "lr_scheduler": lr_scheduler,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
        "lr_warmup_steps": lr_warmup_steps,
        "optimizer_type": "AdamW8bit",
      },
      "training_arguments": {
        "max_train_epochs": max_train_epochs,
        "save_every_n_epochs": save_every_n_epochs,
        "train_batch_size": train_batch_size,
        "noise_offset": None,
        "clip_skip": 2,
        "seed": 42,
        "max_token_length": 225,
        "xformers": True,
        "lowram": True,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "save_precision": "fp16",
        "mixed_precision": "fp16",
        "output_dir": output_folder,
        "logging_dir": logging_dir,
        "output_name": project_name,
        "log_prefix": project_name,
      },
      "model_arguments": {
        "pretrained_model_name_or_path": model_file,
      },
      "saving_arguments": {
        "save_model_as": "safetensors"
      },
      "dreambooth_arguments": {
        "prior_loss_weight": 1.0,
      },
      "dataset_arguments": {
        "cache_latents": True,
      },
    }

    for key in config_dict:
      if isinstance(config_dict[key], dict):
        config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

    with open(config_file, "w") as f:
      f.write(toml.dumps(config_dict))
    print(f"📄 Configuración guardada en {config_file}")

  if "override_dataset_config_file" in globals() and override_dataset_config_file:
    dataset_config_file = override_dataset_config_file
    print(f"⭕ Usando configuración de archivos propia {dataset_config_file}")
  else:
    dataset_config_dict = {
      "general": {
        "resolution": resolution,
        "shuffle_caption": shuffle_caption,
        "flip_aug": flip_aug,        
        "caption_extension": caption_extension,
        "enable_bucket": True,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
        "min_bucket_reso": 320 if resolution > 640 else 256,
        "max_bucket_reso": 1280 if resolution > 640 else 1024,   
      },
      "datasets": [
        {
          "subsets": [
            {
              "num_repeats": num_repeats,
              "keep_tokens": keep_tokens,
              "image_dir": images_folder
            }
          ]
        }
      ]
    }

    for key in dataset_config_dict:
      if isinstance(dataset_config_dict[key], dict):
        dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

    with open(dataset_config_file, "w") as f:
      f.write(toml.dumps(dataset_config_dict))
    print(f"📄 Configuración de archivos guardada en {dataset_config_file}")

def download_model():
  global old_model_url, model_url, model_file
  real_model_url = model_url.strip()
  
  if real_model_url.lower().endswith((".ckpt", ".safetensors")):
    model_file = f"/content{real_model_url[real_model_url.rfind('/'):]}"
  else:
    model_file = "/content/downloaded_model.safetensors"
    if os.path.exists(model_file):
      !rm "{model_file}"
      
  if m := re.search(r"(?:https?://)?(?:www\.)?huggingface\.co/[^/]+/[^/]+/blob", model_url):
    real_model_url = real_model_url.replace("blob", "resolve")
  elif m := re.search(r"(?:https?://)?(?:www\.)?civitai\.com/models/([0-9]+)", model_url):
    real_model_url = f"https://civitai.com/api/download/models/{m.group(1)}"

  !aria2c "{real_model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{model_file}"

  if model_file.lower().endswith(".safetensors"):
    from safetensors.torch import load_file as load_safetensors
    try:
      test = load_safetensors(model_file)
      del test
    except Exception as e:
      #if "HeaderTooLarge" in str(e):
      new_model_file = os.path.splitext(model_file)[0]+".ckpt"
      !mv "{model_file}" "{new_model_file}"
      model_file = new_model_file
      print(f"El modelo ahora es {os.path.splitext(model_file)[0]}.ckpt")

  if model_file.lower().endswith(".ckpt"):
    from torch import load as load_ckpt
    try:
      test = load_ckpt(model_file)
      del test
    except Exception as e:
      return False
      
  return True

def main():
  global dependencies_installed

  if not os.path.exists('/content/drive'):
    print("📂 Conecctando a Google Drive...")
    drive.mount('/content/drive')
  
  for dir in [deps_dir, repo_dir, main_dir, config_dir, datasets_dir, output_dir, logging_dir, images_folder, output_folder, config_folder]:
    os.makedirs(dir, exist_ok=True)

  if not validate_dataset():
    return
  
  if not dependencies_installed:
    print("\n🏭 Instalando...\n")
    clone_repo()
    #!apt -y update -qq
    #!apt install libunwind8=1.2.1-9build1
    #install_ubuntu_deps("https://huggingface.co/Linaqruf/fast-repo/resolve/main/ram_patch.zip", "ram_patch.zip", deps_dir)
    #%env LD_PRELOAD=libtcmalloc.so
    install_ubuntu_deps("https://huggingface.co/Linaqruf/fast-repo/resolve/main/deb-libs.zip", "deb-libs.zip", deps_dir)
    install_dependencies()
    dependencies_installed = True
    print("\n✅ Instalación completada.")
  else:
    print("✅ Ya se ha realizado la instalación.")

  if old_model_url != model_url or not model_file or not os.path.exists(model_file):
    print("\n🔄 Descargando modelo...")
    if not download_model():
      print("\n💥 Error: El modelo es inválido o incorrecto. Recomiendo un enlace de huggingface o civitai.")
      return
    print()
  else:
    print("\n🔄 El modelo ya está descargado.\n")

  create_config()
  
  print("\n⭐ Abriendo entrenador...\n")
  os.chdir(repo_dir)
  
  !accelerate launch --config_file={accelerate_config} --num_cpu_threads_per_process=1 train_network.py --dataset_config={dataset_config_file} --config_file={config_file}

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    display(Markdown("### ✅ ¡Listo! [Puedes descargar tus Loras desde tu Google Drive](https://drive.google.com/drive/my-drive)"))

main()


## *️⃣ Extras

In [ ]:
#@markdown ### 📂 Extraer datos
#@markdown Es lento subir muchos archivos pequeños, si quieres puedes subir un zip y extraerlo aquí.
zip = "/content/drive/MyDrive/lora_training/datasets/warrior.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/lora_training/datasets/" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Done")




In [ ]:
#@markdown ### 🔢 Contar archivos
#@markdown Google Drive hace imposible contar los archivos en una carpeta, por lo que aquí puedes ver la cantidad de archivos en carpetas y subcarpetas.
carpeta = "/content/drive/MyDrive/lora_training/datasets" #@param {type:"string"}
folder = carpeta

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Conectando a Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
for i, (root, dirs, files) in enumerate(os.walk(folder)):
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images and not captions and not others \
                    else f"{images:>4} images | {captions:>4} text files | {others:>4} other files"
pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))


### 📚 Load custom config file

**WARNING!** Advanced users only. <p>

Changing the `dataset_config_file` will allow you to divide your dataset into subsets, and more.

You can also make one of them a regularisation dataset with `is_reg = true`.

Edit and run the below cell to apply a custom config. Run the cell below it to undo.

In [ ]:
dataset_config = """
[[datasets]]

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/lora_training/datasets/project_name/good_images"
num_repeats = 3

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/lora_training/datasets/project_name/bad_images"
num_repeats = 1

[general]
keep_tokens = 1
resolution = 512
shuffle_caption = true
flip_aug = true
caption_extension = ".txt"
enable_bucket = true
bucket_reso_steps = 64
bucket_no_upscale = false
min_bucket_reso = 256
max_bucket_reso = 1024
"""

with open("/content/dataset_config.toml", "w") as f:
  f.write(dataset_config)

override_dataset_config_file = "/content/dataset_config.toml"
override_config_file = None

In [ ]:
override_dataset_config_file = None
override_config_file = None